# Crop Pest Detection Model - Kaggle Optimized

This notebook trains a MobileNetV3-based pest classifier using transfer learning on Kaggle.

**Dataset:** [Pest Dataset](https://www.kaggle.com/datasets/simranvolunesia/pest-dataset)

**Instructions:**
1. Add the dataset to your Kaggle notebook: Click "+ Add Data" → Search "pest-dataset" → Add
2. Enable GPU: Settings → Accelerator → GPU T4 x2
3. Run all cells sequentially

In [ ]:
# Install required packages
!pip install -q tensorflow numpy pandas matplotlib seaborn scikit-learn pillow

print("✓ Packages installed successfully")

## 2. Setup Directories

Configure paths for Kaggle environment.

In [ ]:
import os
from pathlib import Path

# Kaggle dataset path
KAGGLE_INPUT = '/kaggle/input/pest-dataset/pest'

# Working directory paths
WORKING_DIR = '/kaggle/working'
MODELS_DIR = os.path.join(WORKING_DIR, 'models')

# Dataset paths (using Kaggle input)
TRAIN_DIR = os.path.join(KAGGLE_INPUT, 'train')
TEST_DIR = os.path.join(KAGGLE_INPUT, 'test')

# Create models directory
os.makedirs(MODELS_DIR, exist_ok=True)

# Verify dataset structure
print("="*60)
print("KAGGLE ENVIRONMENT SETUP")
print("="*60)
print(f"Kaggle Input: {KAGGLE_INPUT}")
print(f"Train Directory: {TRAIN_DIR}")
print(f"Test Directory: {TEST_DIR}")
print(f"Models Directory: {MODELS_DIR}")
print()

# Check if dataset exists
if os.path.exists(TRAIN_DIR) and os.path.exists(TEST_DIR):
    print("✓ Dataset found successfully!")
    print(f"  Train classes: {len(os.listdir(TRAIN_DIR))}")
    print(f"  Test classes: {len(os.listdir(TEST_DIR))}")
else:
    print("⚠ Dataset not found!")
    print("\nPlease add the dataset:")
    print("  1. Click '+ Add Data' button")
    print("  2. Search for 'pest-dataset' by simranvolunesia")
    print("  3. Click 'Add' to attach it to this notebook")
    raise FileNotFoundError("Dataset not found. Please add the pest-dataset to your Kaggle notebook.")

print("="*60)

## 3. Data Exploration

Explore the dataset structure and visualize sample images.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Count images per class
def count_images_in_dir(directory):
    counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            image_files = [f for f in os.listdir(class_path) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
            counts[class_name] = len(image_files)
    return counts

train_counts = count_images_in_dir(TRAIN_DIR)
test_counts = count_images_in_dir(TEST_DIR)

print("\n" + "="*60)
print("DATASET SUMMARY")
print("="*60)
print(f"\nTotal classes: {len(train_counts)}")
print(f"Total training images: {sum(train_counts.values())}")
print(f"Total test images: {sum(test_counts.values())}")
print(f"\nClass distribution:")
for class_name in sorted(train_counts.keys()):
    print(f"  {class_name:20s}: {train_counts[class_name]:4d} train, {test_counts.get(class_name, 0):4d} test")
print("="*60)

In [ ]:
# Visualize class distribution
import pandas as pd
import seaborn as sns

plt.figure(figsize=(14, 6))

# Create dataframe
df_counts = pd.DataFrame({
    'Class': list(train_counts.keys()),
    'Train': list(train_counts.values()),
    'Test': [test_counts.get(c, 0) for c in train_counts.keys()]
})

# Plot
df_counts_sorted = df_counts.sort_values('Train', ascending=False)
x = np.arange(len(df_counts_sorted))
width = 0.35

plt.bar(x - width/2, df_counts_sorted['Train'], width, label='Train', alpha=0.8)
plt.bar(x + width/2, df_counts_sorted['Test'], width, label='Test', alpha=0.8)

plt.xlabel('Pest Class', fontsize=12)
plt.ylabel('Number of Images', fontsize=12)
plt.title('Dataset Distribution by Class', fontsize=14, fontweight='bold')
plt.xticks(x, df_counts_sorted['Class'], rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
# Visualize sample images from each class
class_names = sorted(train_counts.keys())
num_classes = len(class_names)

fig, axes = plt.subplots(3, min(5, num_classes), figsize=(15, 9))
fig.suptitle('Sample Images from Each Class', fontsize=16, fontweight='bold')

for idx, class_name in enumerate(class_names[:15]):  # Show max 15 classes
    row = idx // 5
    col = idx % 5
    
    class_dir = os.path.join(TRAIN_DIR, class_name)
    images = [f for f in os.listdir(class_dir) 
             if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]
    
    if images:
        img_path = os.path.join(class_dir, images[0])
        img = Image.open(img_path)
        
        if num_classes <= 5:
            axes[col].imshow(img)
            axes[col].set_title(class_name, fontsize=10)
            axes[col].axis('off')
        else:
            axes[row, col].imshow(img)
            axes[row, col].set_title(class_name, fontsize=10)
            axes[row, col].axis('off')

plt.tight_layout()
plt.show()

## 4. Data Preprocessing

Set up data generators with augmentation for training.

In [ ]:
# Define DataPreprocessor for Kaggle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

class DataPreprocessor:
    def __init__(self, img_size=(224, 224), batch_size=32):
        self.img_size = img_size
        self.batch_size = batch_size
    
    def create_train_generator(self):
        return ImageDataGenerator(
            preprocessing_function=preprocess_input,
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest',
            validation_split=0.2
        )
    
    def create_test_generator(self):
        return ImageDataGenerator(preprocessing_function=preprocess_input)
    
    def load_train_data(self, train_dir):
        train_datagen = self.create_train_generator()
        train_gen = train_datagen.flow_from_directory(
            train_dir, target_size=self.img_size, batch_size=self.batch_size,
            class_mode='categorical', subset='training', shuffle=True
        )
        val_gen = train_datagen.flow_from_directory(
            train_dir, target_size=self.img_size, batch_size=self.batch_size,
            class_mode='categorical', subset='validation', shuffle=True
        )
        return train_gen, val_gen
    
    def load_test_data(self, test_dir):
        test_datagen = self.create_test_generator()
        return test_datagen.flow_from_directory(
            test_dir, target_size=self.img_size, batch_size=self.batch_size,
            class_mode='categorical', shuffle=False
        )

print("✓ DataPreprocessor ready")

In [ ]:
# Initialize preprocessor
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

preprocessor = DataPreprocessor(img_size=IMG_SIZE, batch_size=BATCH_SIZE)

# Load data
print("Loading training and validation data...")
train_generator, validation_generator = preprocessor.load_train_data(TRAIN_DIR)

print("\nLoading test data...")
test_generator = preprocessor.load_test_data(TEST_DIR)

# Get class information
class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)

print(f"\n✓ Data loaded successfully!")
print(f"  Classes: {num_classes}")
print(f"  Training samples: {train_generator.samples}")
print(f"  Validation samples: {validation_generator.samples}")
print(f"  Test samples: {test_generator.samples}")
print(f"\n  Class names: {class_names}")

In [ ]:
# Visualize augmented images
print("Visualizing data augmentation...")

# Get a batch of images
sample_batch = next(train_generator)
sample_images = sample_batch[0][:9]
sample_labels = sample_batch[1][:9]

# Denormalize images for display
def denormalize_image(img):
    img = img.copy()
    img += 1
    img *= 127.5
    return np.clip(img, 0, 255).astype(np.uint8)

fig, axes = plt.subplots(3, 3, figsize=(12, 12))
fig.suptitle('Augmented Training Images', fontsize=16, fontweight='bold')

for i, ax in enumerate(axes.flat):
    if i < len(sample_images):
        img = denormalize_image(sample_images[i])
        label_idx = np.argmax(sample_labels[i])
        label = class_names[label_idx]
        
        ax.imshow(img)
        ax.set_title(label, fontsize=11)
        ax.axis('off')

plt.tight_layout()
plt.show()

## 5. Model Development

Build MobileNetV3-based classifier with transfer learning.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV3Large

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Build model
def build_mobilenet_classifier(num_classes, img_size=(224, 224)):
    """
    Build MobileNetV3-based classifier.
    """
    # Load MobileNetV3 with ImageNet weights
    base_model = MobileNetV3Large(
        include_top=False,
        weights='imagenet',
        input_shape=(*img_size, 3),
        include_preprocessing=False
    )
    
    # Freeze base model
    base_model.trainable = False
    
    # Build model
    inputs = keras.Input(shape=(*img_size, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = keras.Model(inputs, outputs, name='crop_pest_detector')
    
    return model

# Build model
print(f"\nBuilding model for {num_classes} classes...")
model = build_mobilenet_classifier(num_classes, IMG_SIZE)

print("\n" + "="*60)
print("MODEL ARCHITECTURE")
print("="*60)
model.summary()
print("="*60)

In [ ]:
# Compile model
LEARNING_RATE = 0.001

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

print("✓ Model compiled successfully!")

## 6. Model Training

Train the model with callbacks for early stopping and model checkpointing.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Training configuration
EPOCHS = 50
MODEL_SAVE_PATH = os.path.join(MODELS_DIR, 'crop_pest_model.h5')

# Callbacks
callbacks = [
    ModelCheckpoint(
        MODEL_SAVE_PATH,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print("="*60 + "\n")

# Train model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("TRAINING COMPLETED")
print("="*60)

## 7. Model Evaluation

Evaluate the trained model and visualize results.

In [ ]:
# Plot training history
def plot_training_history(history):
    """
    Plot training and validation metrics.
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Accuracy
    axes[0, 0].plot(history.history['accuracy'], label='Train', linewidth=2)
    axes[0, 0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    axes[0, 0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(alpha=0.3)
    
    # Loss
    axes[0, 1].plot(history.history['loss'], label='Train', linewidth=2)
    axes[0, 1].plot(history.history['val_loss'], label='Validation', linewidth=2)
    axes[0, 1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(alpha=0.3)
    
    # Precision
    axes[1, 0].plot(history.history['precision'], label='Train', linewidth=2)
    axes[1, 0].plot(history.history['val_precision'], label='Validation', linewidth=2)
    axes[1, 0].set_title('Model Precision', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(alpha=0.3)
    
    # Recall
    axes[1, 1].plot(history.history['recall'], label='Train', linewidth=2)
    axes[1, 1].plot(history.history['val_recall'], label='Validation', linewidth=2)
    axes[1, 1].set_title('Model Recall', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

In [ ]:
# Evaluate on test set
print("\nEvaluating on test set...")
test_loss, test_acc, test_top3, test_precision, test_recall = model.evaluate(test_generator)

print("\n" + "="*60)
print("TEST SET PERFORMANCE")
print("="*60)
print(f"Test Accuracy:       {test_acc*100:.2f}%")
print(f"Test Top-3 Accuracy: {test_top3*100:.2f}%")
print(f"Test Precision:      {test_precision*100:.2f}%")
print(f"Test Recall:         {test_recall*100:.2f}%")
print(f"Test Loss:           {test_loss:.4f}")
print("="*60)

In [ ]:
# Generate predictions for confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

print("\nGenerating predictions for test set...")
test_generator.reset()
predictions = model.predict(test_generator, verbose=1)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Classification report
print("\n" + "="*60)
print("CLASSIFICATION REPORT")
print("="*60)
print(classification_report(y_true, y_pred, target_names=class_names))
print("="*60)

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Per-class performance
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred)

# Create dataframe
performance_df = pd.DataFrame({
    'Class': class_names,
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1,
    'Support': support
})

print("\n" + "="*80)
print("PER-CLASS PERFORMANCE")
print("="*80)
print(performance_df.to_string(index=False))
print("="*80)

# Visualize per-class metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['Precision', 'Recall', 'F1-Score']
colors = ['#3498db', '#2ecc71', '#e74c3c']

for idx, (metric, color) in enumerate(zip(metrics, colors)):
    performance_df_sorted = performance_df.sort_values(metric, ascending=True)
    axes[idx].barh(performance_df_sorted['Class'], performance_df_sorted[metric], color=color, alpha=0.7)
    axes[idx].set_xlabel(metric, fontsize=12)
    axes[idx].set_title(f'{metric} by Class', fontsize=14, fontweight='bold')
    axes[idx].grid(axis='x', alpha=0.3)
    axes[idx].set_xlim([0, 1])

plt.tight_layout()
plt.show()

## 8. Fine-tuning

Fine-tune the model by unfreezing some base layers.

In [ ]:
# Fine-tuning configuration
FINE_TUNE_EPOCHS = 20
FINE_TUNE_LR = 1e-5
UNFREEZE_LAYERS = 50
FINE_TUNED_MODEL_PATH = os.path.join(MODELS_DIR, 'crop_pest_model_finetuned.h5')

print("\n" + "="*60)
print("FINE-TUNING MODEL")
print("="*60)
print(f"Unfreezing last {UNFREEZE_LAYERS} layers of base model")
print(f"Fine-tuning learning rate: {FINE_TUNE_LR}")
print("="*60 + "\n")

# Unfreeze base model layers
base_model = model.layers[1]  # MobileNetV3 is the second layer
base_model.trainable = True

# Freeze all except last UNFREEZE_LAYERS
for layer in base_model.layers[:-UNFREEZE_LAYERS]:
    layer.trainable = False

# Recompile
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=FINE_TUNE_LR),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

# Callbacks for fine-tuning
fine_tune_callbacks = [
    ModelCheckpoint(
        FINE_TUNED_MODEL_PATH,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-8,
        verbose=1
    )
]

# Fine-tune
history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=FINE_TUNE_EPOCHS,
    callbacks=fine_tune_callbacks,
    verbose=1
)

print("\n" + "="*60)
print("FINE-TUNING COMPLETED")
print("="*60)

In [ ]:
# Evaluate fine-tuned model
print("\nEvaluating fine-tuned model on test set...")
test_loss_ft, test_acc_ft, test_top3_ft, test_precision_ft, test_recall_ft = model.evaluate(test_generator)

print("\n" + "="*60)
print("FINE-TUNED MODEL PERFORMANCE")
print("="*60)
print(f"Test Accuracy:       {test_acc_ft*100:.2f}% (was {test_acc*100:.2f}%)")
print(f"Test Top-3 Accuracy: {test_top3_ft*100:.2f}% (was {test_top3*100:.2f}%)")
print(f"Test Precision:      {test_precision_ft*100:.2f}% (was {test_precision*100:.2f}%)")
print(f"Test Recall:         {test_recall_ft*100:.2f}% (was {test_recall*100:.2f}%)")
print(f"Test Loss:           {test_loss_ft:.4f} (was {test_loss:.4f})")
print("="*60)

# Plot fine-tuning history
plot_training_history(history_fine)

## 9. Save Model and Export

Save the final model for download and deployment.

## 10. Sample Prediction

Test the model with a sample prediction.

In [ ]:
import json
from datetime import datetime

# Save class names
class_names_path = os.path.join(MODELS_DIR, 'class_names.json')
with open(class_names_path, 'w') as f:
    json.dump(class_names, f, indent=2)
print(f"✓ Class names saved to: {class_names_path}")

# Save model metadata
metadata = {
    "created_at": datetime.now().isoformat(),
    "model_path": FINE_TUNED_MODEL_PATH,
    "num_classes": num_classes,
    "class_names": class_names,
    "image_size": list(IMG_SIZE),
    "architecture": "MobileNetV3Large",
    "framework": "TensorFlow/Keras",
    "training": {
        "initial_epochs": EPOCHS,
        "fine_tune_epochs": FINE_TUNE_EPOCHS,
        "batch_size": BATCH_SIZE,
        "initial_lr": LEARNING_RATE,
        "fine_tune_lr": FINE_TUNE_LR
    },
    "performance": {
        "test_accuracy": float(test_acc_ft),
        "test_top3_accuracy": float(test_top3_ft),
        "test_precision": float(test_precision_ft),
        "test_recall": float(test_recall_ft),
        "test_loss": float(test_loss_ft)
    }
}

metadata_path = os.path.join(MODELS_DIR, 'model_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✓ Model metadata saved to: {metadata_path}")

# Save model summary
summary_path = os.path.join(MODELS_DIR, 'model_summary.txt')
with open(summary_path, 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))
print(f"✓ Model summary saved to: {summary_path}")

print("\n" + "="*60)
print("ALL FILES SAVED SUCCESSFULLY")
print("="*60)
print(f"\nModel files:")
print(f"  - Initial model: {MODEL_SAVE_PATH}")
print(f"  - Fine-tuned model: {FINE_TUNED_MODEL_PATH}")
print(f"  - Class names: {class_names_path}")
print(f"  - Metadata: {metadata_path}")
print(f"  - Model summary: {summary_path}")
print("="*60)

In [ ]:
# Sample prediction on a test image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input

def predict_image(model, image_path, class_names, top_k=3):
    """
    Predict pest class for a single image.
    """
    # Load and preprocess image
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    # Predict
    predictions = model.predict(img_array, verbose=0)
    
    # Get top K predictions
    top_indices = np.argsort(predictions[0])[-top_k:][::-1]
    top_probs = predictions[0][top_indices]
    
    # Display
    plt.figure(figsize=(10, 4))
    
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"Predicted: {class_names[top_indices[0]]}\nConfidence: {top_probs[0]*100:.2f}%",
             fontsize=12, fontweight='bold')
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    colors = ['#2ecc71' if i == 0 else '#3498db' for i in range(top_k)]
    plt.barh([class_names[i] for i in top_indices], top_probs, color=colors, alpha=0.7)
    plt.xlabel('Confidence', fontsize=11)
    plt.title('Top Predictions', fontsize=12, fontweight='bold')
    plt.xlim([0, 1])
    plt.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print results
    print("\nPrediction Results:")
    for i, (idx, prob) in enumerate(zip(top_indices, top_probs), 1):
        print(f"  {i}. {class_names[idx]:20s} - {prob*100:6.2f}%")

# Get a random test image
import random
test_class = random.choice(class_names)
test_class_dir = os.path.join(TEST_DIR, test_class)
test_images = [f for f in os.listdir(test_class_dir) 
              if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

if test_images:
    sample_image = os.path.join(test_class_dir, random.choice(test_images))
    print(f"\nTesting with image from class: {test_class}")
    print(f"Image path: {sample_image}\n")
    predict_image(model, sample_image, class_names)
else:
    print("No test images available for prediction demo")

## Summary

This notebook has successfully:

1. ✓ Set up Kaggle environment and verified dataset
2. ✓ Explored and visualized the Pest Dataset
3. ✓ Implemented data preprocessing with augmentation
4. ✓ Built a MobileNetV3-based classifier using transfer learning
5. ✓ Trained the model with early stopping and checkpointing
6. ✓ Evaluated the model with comprehensive metrics
7. ✓ Fine-tuned the model for improved performance
8. ✓ Saved the model with metadata for deployment

**Download Your Model:**
All files are saved in `/kaggle/working/models/`:
- `crop_pest_model_finetuned.h5` - Fine-tuned model
- `class_names.json` - Class labels
- `model_metadata.json` - Model information

To download: Click on the **Output** tab → Download the models folder